In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
!wget -c https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
!chmod +x Anaconda3-2019.10-Linux-x86_64.sh
!bash ./Anaconda3-2019.10-Linux-x86_64.sh -b -f -p /usr/local
!conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
#import deepchem as dc

--2020-07-24 06:32:59--  https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 530308481 (506M) [application/x-sh]
Saving to: ‘Anaconda3-2019.10-Linux-x86_64.sh’

Anaconda3-2019.10-L 100%[===================>] 505.74M   100MB/s    in 5.3s    

2020-07-24 06:33:04 (96.1 MB/s) - ‘Anaconda3-2019.10-Linux-x86_64.sh’ saved [530308481/530308481]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _ipyw_jlab_nb_ext_conf==0.1.0=py37_0
    - _libgcc_mutex==0.1=main
    - alabaster==0.7.12=py37_0
    - anaconda-client==1.7.2=py37_0
    - anaconda-navigator==1.9.7=py3

In [4]:
import deepchem as dc

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
dc.__version__

'2.3.0'

In [6]:
# libraries are installed
#now code has to be made

In [7]:
import numpy as np

In [8]:
#Training a Model to Predict Toxicity of Molecules

In [9]:
#getting the data for the objective
tox21_tasks, tox21_datasets, transformers = dc.molnet.load_tox21()

Loading raw samples now.
shard_size: 8192
About to start loading CSV from /tmp/tox21.csv.gz
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
TIMING: featurizing shard 0 took 31.512 s
TIMING: dataset construction took 31.837 s
Loading dataset from disk.
TIMING: dataset construction took 0.393 s
Loading dataset from disk.
TIMING: dataset construction took 0.192 s
Loading dataset from disk.
TIMING: dataset construction took 0.187 s
Loading dataset from disk.
TIMING: dataset construction took 0.314 s
Loading dataset from disk.
TIMING: dataset construction took 0.043 s
Loading dataset from disk.
TIMING: dataset construction took 0.042 s
Loading dataset from disk.


In [10]:
#corresponds to enzymatic assays
tox21_tasks

['NR-AR',
 'NR-AR-LBD',
 'NR-AhR',
 'NR-Aromatase',
 'NR-ER',
 'NR-ER-LBD',
 'NR-PPAR-gamma',
 'SR-ARE',
 'SR-ATAD5',
 'SR-HSE',
 'SR-MMP',
 'SR-p53']

In [11]:
# the dataset, split into train valid test
tox21_datasets

(<deepchem.data.datasets.DiskDataset at 0x7f7e6fc12748>,
 <deepchem.data.datasets.DiskDataset at 0x7f7e6fee4f28>)

In [12]:
train_dataset, valid_dataset, test_dataset = tox21_datasets

In [13]:
#A transformer is an object that modifies a dataset in some way. Deep‐
#Chem provides many transformers that manipulate data in useful ways.

In [14]:
transformers

In [15]:
# """
# DeepChem’s dc.models submodule contains a variety of
# different life science–specific models. All of these various models inherit from the
# parent class dc.models.Model


# dc.models.MultitaskClassifier . This model builds a fully connected
# network (an MLP) that maps input features to multiple output predictions. This
# makes it useful for multitask problems, where there are multiple labels for every sam‐
# ple. It’s well suited for our Tox21 datasets, since we have a total of 12 different assays
# we wish to predict simultaneously.


#I have changed the code compared to the book number of layers, 2 layer with 512 nodes each
# """


model = dc.models.MultitaskClassifier(n_tasks=12,n_features=1024,layer_sizes=[512,512])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [16]:
model.fit(train_dataset, nb_epoch=20)

0.035867755425473054

In [17]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)

In [18]:
train_scores = model.evaluate(train_dataset, [metric], transformers)

computed_metrics: [0.9994246504649034, 0.9998404383429673, 0.996804978312666, 0.9979961509279627, 0.9814226241268363, 0.9982318117533349, 0.9991721787447807, 0.9900934782790622, 0.998744197295435, 0.9986589872144058, 0.9953338526449739, 0.9987369245399116]


In [19]:
train_scores

{'mean-roc_auc_score': 0.9962050227206033}

In [20]:
test_scores = model.evaluate(test_dataset, [metric], transformers)

computed_metrics: [0.7352702453421878, 0.8587506322711178, 0.8160145004797954, 0.7732363407996651, 0.7024989651190658, 0.7882966299632966, 0.7415797749089706, 0.6734977912008655, 0.8177263772841519, 0.6872219017798287, 0.841151895576987, 0.722630266075388]


In [21]:
test_scores

{'mean-roc_auc_score': 0.7631562767334433}

In [ ]:
#model is heaviy overfit , train aucroc is 0.996, testing is 0.762. one layer extra with more epochs 
# didn't help. This is as expected acording to the theory in ML

In [ ]:
# next up is the CNN example for MNIST

In [22]:
!mkdir MNIST_data

In [ ]:
!pwd

/content


In [23]:
cd MNIST_data

/content/MNIST_data


In [24]:
!wget http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz

--2020-07-24 06:49:27--  http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 172.67.171.76, 104.28.6.204, 104.28.7.204, ...
Connecting to yann.lecun.com (yann.lecun.com)|172.67.171.76|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9912422 (9.5M) [application/x-gzip]
Saving to: ‘train-images-idx3-ubyte.gz’

train-images-idx3-u 100%[===================>]   9.45M  9.10MB/s    in 1.0s    

2020-07-24 06:49:29 (9.10 MB/s) - ‘train-images-idx3-ubyte.gz’ saved [9912422/9912422]

--2020-07-24 06:49:31--  http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 104.28.7.204, 172.67.171.76, 104.28.6.204, ...
Connecting to yann.lecun.com (yann.lecun.com)|104.28.7.204|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28881 (28K) [application/x-gzip]
Saving to: ‘train-labels-idx1-ubyte.gz’

train-labels-idx1-u 100%[===================>]  28.20K  --.-K

In [ ]:
#cd MNIST_data/

/content/MNIST_data


In [25]:
!pwd

/content/MNIST_data


In [26]:
cd ..

/content


In [27]:
from tensorflow.examples.tutorials.mnist import input_data

In [28]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [29]:
import tensorflow as tf
import deepchem.models.tensorgraph.layers as layers

In [30]:
train_dataset = dc.data.NumpyDataset(mnist.train.images, mnist.train.labels)
test_dataset = dc.data.NumpyDataset(mnist.test.images, mnist.test.labels)

In [66]:
#del model

In [67]:
#model = dc.models.TensorGraph(model_dir='mnist')
#feature = layers.Feature(shape=(None, 784))
#label = layers.Label(shape=(None, 10))
#make_image = layers.Reshape(shape=(None, 28, 28), in_layers=feature)
#conv2d_1 = layers.Conv2D(num_outputs=32, activation_fn=tf.nn.relu,in_layers=make_image)
#conv2d_2 = layers.Conv2D(num_outputs=64, activation_fn=tf.nn.relu,in_layers=conv2d_1)
#flatten = layers.Flatten(in_layers=conv2d_2)
#dense1 = layers.Dense(out_channels=1024, activation_fn=tf.nn.relu,in_layers=flatten)
#dense2 = layers.Dense(out_channels=10, activation_fn=None, in_layers=dense1)
#smce = layers.SoftMaxCrossEntropy(in_layers=[label, dense2])
#loss = layers.ReduceMean(in_layers=smce)
#model.set_loss(loss)
#output = layers.SoftMax(in_layers=dense2)
#model.add_output(output)

In [69]:
from tensorflow.keras.layers import Reshape, Conv2D, Flatten, Dense, Softmax

In [70]:
model = tf.keras.Sequential([
    Reshape((28, 28, 1)),
    Conv2D(filters=32, kernel_size=5, activation=tf.nn.relu),
    Conv2D(filters=64, kernel_size=5, activation=tf.nn.relu),
    Flatten(),
    Dense(1024, activation=tf.nn.relu),
    Dense(10),
    Softmax()
])

In [72]:
model = dc.models.KerasModel(model, dc.models.losses.CategoricalCrossEntropy())

In [73]:
model.fit(train_dataset, nb_epoch=10)

0.0

In [100]:
prediction = np.squeeze(model.predict_on_batch(test_dataset.X))

In [108]:
prediction.shape

(10000, 10)

In [109]:
y_classes = prediction.argmax(axis=-1) 

In [110]:
y_classes

array([7, 2, 1, ..., 4, 5, 6])

In [113]:
y_orig = test_dataset.y.argmax(axis=-1)

In [114]:
y_orig

array([7, 2, 1, ..., 4, 5, 6])

In [104]:
#ypred = np.argmax(prediction, axis=1)

In [105]:
#ypred

In [90]:
#from sklearn import preprocessing
#lb = preprocessing.LabelBinarizer()

In [81]:
from sklearn.metrics import accuracy_score

In [82]:
from sklearn.metrics import roc_curve, auc
import numpy as np

print("Validation")
prediction = np.squeeze(model.predict_on_batch(test_dataset.X))

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(10):
    fpr[i], tpr[i], thresh = roc_curve(test_dataset.y[:, i], prediction[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    print("class %s:auc=%s" % (i, roc_auc[i]))

Validation
class 0:auc=0.9999884610163355
class 1:auc=0.9999847939354636
class 2:auc=0.9999653159554385
class 3:auc=0.999926981574687
class 4:auc=0.9999699627673823
class 5:auc=0.9998927912385701
class 6:auc=0.9999287712779933
class 7:auc=0.999955872225046
class 8:auc=0.9999457419610516
class 9:auc=0.9998998006926649


In [98]:
test_dataset.y[:, i].shape

(10000,)

In [99]:
ypred.shape

(10000,)

In [115]:
accuracy_score(y_classes, ypred)

1.0

In [74]:
#metric = dc.metrics.Metric(dc.metrics.accuracy_score)

In [120]:
#train_dataset.get_shape()

In [121]:
#train_scores = model.evaluate(train_dataset, [metric])
#test_scores = model.evaluate(test_dataset, [metric])

In [122]:
#train_scores

In [116]:
#for training
y_train = np.squeeze(model.predict_on_batch(train_dataset.X))

In [125]:
y_train_predlabels = y_train.argmax(axis=-1)

In [119]:
y_train_labels.shape

(55000,)

In [123]:
y_train_orig = train_dataset.y.argmax(axis=-1)

In [126]:
#
training_accuracy = accuracy_score(y_train_orig, y_train_predlabels)

In [127]:
training_accuracy

0.9992181818181818

In [124]:
# test accuracy already calculated above
accuracy_score(y_classes, ypred)

1.0

In [128]:
########### completed after running a GPU instance on colab